In [1]:
#Environment Setup
!pip install mlx
!pip install torch
!pip install numpy
!pip install mnist
!pip install scikit-learn

In [2]:
#Library Import
import mlx.core as mx
import mlx.nn as nn
import mlx.optimizers as optim
import torch
import numpy as np

## 1. 数组创建和基本操作

### 数组创建:

PyTorch: 使用 torch.arange 创建数组，并使用 reshape 进行重塑。
MLX: 使用 mx.arange 创建数组，并使用 reshape 进行重塑。

### 基本操作:

PyTorch: 可以直接对张量进行加法操作，结果会立即计算并存储。
MLX: 也可以直接对数组进行加法操作，但由于 MLX 支持懒惰求值，操作结果需要显式调用 mx.eval 进行求值。

### PyTorch 

In [3]:
x_t = torch.arange(12, dtype=torch.float32).reshape((3, 4))
print(x_t)
print(x_t.shape)

# 基本操作
y_t = x_t + 2
print(y_t)

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.]])
torch.Size([3, 4])
tensor([[ 2.,  3.,  4.,  5.],
        [ 6.,  7.,  8.,  9.],
        [10., 11., 12., 13.]])


### MLX 

In [4]:
#MLX Code
x_m = mx.arange(12, dtype=mx.float32).reshape((3, 4))
print(x_m)
print(x_m.shape)

# 基本操作
y_m = x_m + 2

# y_m = x_m + 2的操作已经执行
print(y_m)

#懒惰求值（lazy evaluation）
mx.eval(y_m) 
print(y_m)

array([[0, 1, 2, 3],
       [4, 5, 6, 7],
       [8, 9, 10, 11]], dtype=float32)
(3, 4)
array([[2, 3, 4, 5],
       [6, 7, 8, 9],
       [10, 11, 12, 13]], dtype=float32)
array([[2, 3, 4, 5],
       [6, 7, 8, 9],
       [10, 11, 12, 13]], dtype=float32)


## 2.数据类型转换

### 转换为 NumPy 数组:

PyTorch: 使用 .numpy() 方法将张量转换为 NumPy 数组。
MLX: 使用 np.array() 将 MLX 数组转换为 NumPy 数组。

### 从 NumPy 数组转换为框架张量:

PyTorch: 使用 torch.tensor() 方法将 NumPy 数组转换为 PyTorch 张量。
MLX: 使用 mx.array() 方法将 NumPy 数组转换为 MLX 张量。

### PyTorch 

In [5]:
# 转换为 NumPy 数组
x_np = x_t.numpy()
print(x_np)

# 从 NumPy 数组转换为 PyTorch 张量
y_np = np.array([[1, 2], [3, 4]], dtype=np.float32)
y_t = torch.tensor(y_np)
print(y_t)


[[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]
 [ 8.  9. 10. 11.]]
tensor([[1., 2.],
        [3., 4.]])


### MLX 

In [6]:
# 转换为 NumPy 数组
x_np = np.array(x_m)
print(x_np)

# 从 NumPy 数组转换为 MLX 张量
y_np = np.array([[1, 2], [3, 4]], dtype=np.float32)
y_m = mx.array(y_np)
print(y_m)

[[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]
 [ 8.  9. 10. 11.]]
array([[1, 2],
       [3, 4]], dtype=float32)


## 3. 设备管理（CPU/GPU）统一内存

### 设备检查和选择:

PyTorch: 使用 torch.backends.mps.is_available() 检查 MPS 设备是否可用，并使用 torch.device 选择设备。如果 MPS 不可用，则使用 CPU。
MLX: 没有显式的设备选择，MLX 默认支持统一内存，~~数据自动在需要时移动到 GPU~~。
### 张量创建和移动:

PyTorch: 使用 torch.arange 创建张量，并使用 .to(device) 将张量移动到指定设备。
MLX: 使用 mx.arange 创建张量，张量自动在统一内存中初始化。
张量的创建有很多方法

### 在设备上执行计算:

PyTorch: 在指定设备上执行计算，并使用 .to("cpu") 将结果移动回 CPU 以进行打印。
MLX: 使用 mx.add 在 GPU 上执行计算，并使用 mx.eval 进行懒惰求值，然后打印结果。

### PyTorch

In [7]:
import torch

# Check if MPS is available
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("MPS device is available")
else:
    device = torch.device("cpu")
    print("MPS device is not available, using CPU")

# Create tensor and move it to the appropriate device
x = torch.arange(12, dtype=torch.float32).reshape((3, 4))
x = x.to(device)

# Perform computation on the device
result = (x + x).to("cpu")
print(result)
### 验证，变量位置没有变化？ 
### Tensor创建后在GPU? 

MPS device is available
tensor([[ 0.,  2.,  4.,  6.],
        [ 8., 10., 12., 14.],
        [16., 18., 20., 22.]])


### MLX 

In [8]:
import mlx.core as mx

# 创建张量，自动在统一内存中初始化
x = mx.arange(12, dtype=mx.float32).reshape((3, 4))

# 在 GPU 上执行计算
result = mx.add(x, x, stream=mx.gpu)
mx.eval(result)
print(result)
### 变量在cpu/gpu? 内存位置是否会变化

array([[0, 2, 4, 6],
       [8, 10, 12, 14],
       [16, 18, 20, 22]], dtype=float32)


## 4. 索引和切片
PyTorch 和 MLX 在索引和切片方面的使用方式基本相同。

PyTorch: 使用标准的 Python 切片和索引语法，直接对张量进行操作。

MLX: 使用相同的标准 Python 切片和索引语法，直接对数组进行操作。

### PyTorch 

In [9]:
import torch

x_t = torch.arange(12, dtype=torch.float32).reshape((3, 4))

# 索引和切片
print(x_t[1, 2])
print(x_t[:, 1:3])


tensor(6.)
tensor([[ 1.,  2.],
        [ 5.,  6.],
        [ 9., 10.]])


### MLX 

In [10]:
import mlx.core as mx

x_m = mx.arange(12, dtype=mx.float32).reshape((3, 4))

# 索引和切片
print(x_m[1, 2])
print(x_m[:, 1:3])


array(6, dtype=float32)
array([[1, 2],
       [5, 6],
       [9, 10]], dtype=float32)


## 5. 保存和加载数据
### 保存数据:

PyTorch: 使用 torch.save(x_t, 'tensor.pt') 将张量保存到文件 'tensor.pt' 中。
MLX: 使用 mx.save('array', x_m) 将数组保存到文件 'array.npy' 中。
### 加载数据:

PyTorch: 使用 torch.load('tensor.pt') 从文件中加载张量。
MLX: 使用 mx.load('array.npy') 从文件中加载数组。

### PyTorch 

In [11]:
import torch

x_t = torch.arange(12, dtype=torch.float32).reshape((3, 4))

# 保存张量
torch.save(x_t, 'tensor.pt')

# 加载张量
loaded_x_t = torch.load('tensor.pt')
print(loaded_x_t)


tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.]])


### MLX 

In [12]:
import mlx.core as mx

x_m = mx.arange(12, dtype=mx.float32).reshape((3, 4))

# 保存数组
mx.save('array', x_m)

# 加载数组
loaded_x_m = mx.load('array.npy')
print(loaded_x_m)


array([[0, 1, 2, 3],
       [4, 5, 6, 7],
       [8, 9, 10, 11]], dtype=float32)


## 6. 梯度计算

### 创建张量和设置梯度计算:

PyTorch: 使用 torch.tensor 创建张量，并设置 requires_grad=True 以启用自动梯度计算。
MLX: 直接使用 mx.array 创建数组，无需显式设置梯度计算，梯度计算通过函数实现。
### 定义损失函数:

PyTorch: 直接在张量上进行操作，并定义损失函数。
MLX: 定义一个损失函数，该函数接受数组作为输入并返回损失值。
### 梯度计算:

PyTorch: 使用 backward() 方法进行反向传播，计算梯度。
MLX: 使用 mx.grad 函数获取梯度函数，通过该函数计算梯度。

### PyTorch 

In [13]:
import torch

# 创建张量并设置需要计算梯度
x_t = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)

# 定义损失函数
loss = x_t.pow(2).sum()

# 反向传播
loss.backward()
print("Gradient:", x_t.grad)


Gradient: tensor([2., 4., 6.])


### MLX 

In [14]:
import mlx.core as mx

# 定义损失函数
def loss_fn(x):
    return mx.sum(mx.square(x))

# 获取梯度函数
grad_fn = mx.grad(loss_fn)

# 计算梯度
x_m = mx.array([1.0, 2.0, 3.0])
grad_m = grad_fn(x_m)
print("Gradient:", grad_m)


Gradient: array([2, 4, 6], dtype=float32)


## 7. 优化器
PyTorch 优化器直接管理参数更新，自动处理梯度清零和参数更新。

MLX 需要手动传递梯度和学习率来更新参数，优化器不直接管理参数。

### PyTorch 

In [15]:
import torch
import torch.optim as optim

# 创建模型参数
params = [torch.tensor([1.0, 2.0, 3.0], requires_grad=True)]

# 创建优化器
optimizer = optim.SGD(params, lr=0.1)

# 定义损失函数
def closure():
    optimizer.zero_grad()
    loss = params[0].pow(2).sum()
    loss.backward()
    return loss

# 执行优化步骤
for _ in range(10):
    optimizer.step(closure)
    print(params[0])


tensor([0.8000, 1.6000, 2.4000], requires_grad=True)
tensor([0.6400, 1.2800, 1.9200], requires_grad=True)
tensor([0.5120, 1.0240, 1.5360], requires_grad=True)
tensor([0.4096, 0.8192, 1.2288], requires_grad=True)
tensor([0.3277, 0.6554, 0.9830], requires_grad=True)
tensor([0.2621, 0.5243, 0.7864], requires_grad=True)
tensor([0.2097, 0.4194, 0.6291], requires_grad=True)
tensor([0.1678, 0.3355, 0.5033], requires_grad=True)
tensor([0.1342, 0.2684, 0.4027], requires_grad=True)
tensor([0.1074, 0.2147, 0.3221], requires_grad=True)


### MLX

In [16]:
import mlx.core as mx
import mlx.optimizers as optim

# 创建模型参数
params = mx.array([1.0, 2.0, 3.0])

# 定义优化器
optimizer = optim.SGD(learning_rate=0.1)

# 定义损失函数和梯度函数
def loss_fn(params):
    return mx.sum(mx.square(params))

loss_and_grad_fn = mx.value_and_grad(loss_fn)

# 执行优化步骤
for _ in range(10):
    loss, grads = loss_and_grad_fn(params)
    params -= optimizer.learning_rate * grads
    mx.eval(params)
    # print(params)
    print(loss)


array(14, dtype=float32)
array(8.96, dtype=float32)
array(5.7344, dtype=float32)
array(3.67002, dtype=float32)
array(2.34881, dtype=float32)
array(1.50324, dtype=float32)
array(0.962073, dtype=float32)
array(0.615726, dtype=float32)
array(0.394065, dtype=float32)
array(0.252202, dtype=float32)


## 8. 多层感知机（MLP）在 PyTorch 和 MLX 中的实现区别

### a. 模型定义和模块化

**PyTorch**:
- 使用 `nn.Module` 创建模型，并定义网络层。
- 使用 `forward` 方法定义前向传播过程。

**MLX**:
- 使用 `nn.Module` 创建模型，并定义网络层。
- 使用 `__call__` 方法定义前向传播过程。

### b. 数据处理和张量操作

**PyTorch**:
- 使用 `torch.tensor` 创建张量。
- 数据加载和预处理通常使用 `torchvision` 和 `torch.utils.data.DataLoader`。

**MLX**:
- 使用 `mx.array` 创建数组。
- 数据加载和预处理需要手动实现。 mlx-data

### c. 损失函数和优化器

**PyTorch**:
- 使用 `nn.CrossEntropyLoss` 作为损失函数。
- 使用 `optim.SGD` 作为优化器。

**MLX**:
- 需要手动实现交叉熵损失函数。
- 使用 `optim.SGD` 作为优化器。

### d. 训练和评估

**PyTorch**:
- 使用 `zero_grad` 清零梯度，使用 `backward` 计算梯度，使用 `step` 更新参数。
- 训练和评估过程中可以使用 `with torch.no_grad()` 关闭梯度计算，以提高评估速度。

**MLX**:
- 使用 `nn.value_and_grad` 获取损失和梯度。
- 使用 `optimizer.update` 更新参数。
- 每次迭代时重新计算梯度以确保梯度不会累积。
- 需要手动实现批处理迭代器和数据打乱。


## Torch MLP示例

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# 设置随机种子
def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

# 加载数据集
iris = load_iris()
data, targets = iris.data, iris.target

# 数据预处理
scaler = StandardScaler()
data = scaler.fit_transform(data)

# 拆分数据集
train_data, test_data, train_targets, test_targets = train_test_split(data, targets, test_size=0.2, random_state=42)

# 转换为 PyTorch 张量
train_data = torch.tensor(train_data, dtype=torch.float32)
train_targets = torch.tensor(train_targets, dtype=torch.long)
test_data = torch.tensor(test_data, dtype=torch.float32)
test_targets = torch.tensor(test_targets, dtype=torch.long)


In [18]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(4, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 3)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# 实例化模型
model = MLP()


In [19]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
def train_model():
    set_seed(42)
    model = MLP()
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

    for epoch in range(10):
        model.train()
        optimizer.zero_grad()
        output = model(train_data)
        loss = criterion(output, train_targets)
        loss.backward()
        optimizer.step()
        
        print(f'Epoch {epoch+1}: Loss = {loss.item()}')
    
    model.eval()
    correct = 0
    with torch.no_grad():
        output = model(test_data)
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(test_targets.view_as(pred)).sum().item()

    accuracy = correct / len(test_targets)
    print(f'Test Accuracy: {accuracy * 100:.2f}%')

train_model()


Epoch 1: Loss = 1.139323353767395
Epoch 2: Loss = 1.1306538581848145
Epoch 3: Loss = 1.114494800567627
Epoch 4: Loss = 1.0922130346298218
Epoch 5: Loss = 1.0651919841766357
Epoch 6: Loss = 1.034608244895935
Epoch 7: Loss = 1.001425862312317
Epoch 8: Loss = 0.9668508172035217
Epoch 9: Loss = 0.9314693808555603
Epoch 10: Loss = 0.8963340520858765
Test Accuracy: 80.00%


## MLX MLP示例

In [20]:
import numpy as np
import mlx.core as mx
import mlx.nn as nn
import mlx.optimizers as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 设置随机种子
def set_seed(seed):
    np.random.seed(seed)
    mx.random.seed(seed)

set_seed(42)

# 加载数据集
iris = load_iris()
data, targets = iris.data, iris.target

# 数据预处理
scaler = StandardScaler()
data = scaler.fit_transform(data)

# 拆分数据集
train_data, test_data, train_targets, test_targets = train_test_split(data, targets, test_size=0.2, random_state=42)

# 转换为 MLX 数组
train_data = mx.array(train_data)
train_targets = mx.array(train_targets)
test_data = mx.array(test_data)
test_targets = mx.array(test_targets)


In [21]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(4, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 3)

    def __call__(self, x):
        x = mx.maximum(self.fc1(x), 0.0)
        x = mx.maximum(self.fc2(x), 0.0)
        x = self.fc3(x)
        return x

# 实例化模型
model = MLP()


In [22]:
def cross_entropy(predictions, targets):
    log_softmax = mx.log(mx.exp(predictions) / mx.sum(mx.exp(predictions), axis=1, keepdims=True))
    return -mx.mean(mx.sum(log_softmax * targets, axis=1))

def loss_fn(model, X, y):
    predictions = model(X)
    y_one_hot = mx.eye(3)[y]
    return cross_entropy(predictions, y_one_hot)

def eval_fn(model, X, y):
    return mx.mean(mx.argmax(model(X), axis=1) == y)

def train_model():
    set_seed(42)
    model = MLP()
    optimizer = optim.SGD(learning_rate=0.01)
    
    batch_size = 64
    num_epochs = 10

    def batch_iterate(batch_size, X, y):
        perm = mx.array(np.random.permutation(y.size))
        for s in range(0, y.size, batch_size):
            ids = perm[s:s + batch_size]
            yield X[ids], y[ids]

    for epoch in range(num_epochs):
        for X_batch, y_batch in batch_iterate(batch_size, train_data, train_targets):
            loss, grads = nn.value_and_grad(model, loss_fn)(model, X_batch, y_batch)
            optimizer.update(model, grads)
            # mx.eval(model.parameters())

        loss = loss_fn(model, train_data, train_targets)
        print(f'Epoch {epoch+1}: Loss = {loss.item()}')

    accuracy = eval_fn(model, test_data, test_targets).item()
    print(f'Test Accuracy: {accuracy * 100:.2f}%')

train_model()


Epoch 1: Loss = 1.0917575359344482
Epoch 2: Loss = 1.0718941688537598
Epoch 3: Loss = 1.053246021270752
Epoch 4: Loss = 1.0353620052337646
Epoch 5: Loss = 1.0182981491088867
Epoch 6: Loss = 1.0017993450164795
Epoch 7: Loss = 0.986078679561615
Epoch 8: Loss = 0.9708292484283447
Epoch 9: Loss = 0.9563468098640442
Epoch 10: Loss = 0.9421473741531372
Test Accuracy: 73.33%
